In [1]:
import torch
import pandas as pd
import numpy as np
import torch.nn as nn
from sklearn.model_selection import train_test_split 
from sklearn.metrics import accuracy_score, log_loss
from torchsummary import summary
from sklearn.preprocessing import MinMaxScaler, LabelEncoder
import matplotlib.pyplot as plt
import os 
from torch.utils.data import DataLoader
from torch.utils.data import TensorDataset
os.chdir("D:/Training/Academy/ML(Python)/Cases/Glass_Identification")

In [2]:
glass = pd.read_csv("Glass.csv")
dum_gls = pd.get_dummies(glass)
X = dum_gls.iloc[:,:-6]
y = dum_gls.iloc[:,-6:]

In [3]:
type(X)

pandas.core.frame.DataFrame

In [4]:
X.shape, y.shape

((214, 9), (214, 6))

In [5]:
scaler = MinMaxScaler()
X_train, X_test, y_train, y_test = train_test_split(X, y.values, test_size = 0.3, 
                                                    random_state=24,stratify=glass['Type'])
X_scl_trn = scaler.fit_transform(X_train) 
X_scl_tst = scaler.transform(X_test) 

In [6]:
type(y_train)

numpy.ndarray

In [7]:
X_torch = torch.from_numpy(X_scl_trn)
y_torch = torch.from_numpy(y_train)
print(X_torch.size())
print(y_torch.size())

torch.Size([149, 9])
torch.Size([149, 6])


In [8]:
joint_dataset = TensorDataset(X_torch.float(), y_torch.float())
torch.manual_seed(25)
data_loader = DataLoader(dataset=joint_dataset, batch_size=40, shuffle=True)

### Model Definition

In [9]:
torch.manual_seed(25)
model = nn.Sequential(nn.Linear(in_features=X_scl_trn.shape[1],out_features=7),
                      nn.ReLU(),
                      nn.Linear(in_features=7,out_features=6))

In [10]:
torch.manual_seed(25)
class MLPClassifier(torch.nn.Module):    
    def __init__(self, num_features):
        super().__init__()
        self.linear1 = nn.Linear(in_features=num_features, out_features=18)
        self.linear2 = nn.Linear(in_features=18, out_features=7)
        self.linear3 = nn.Linear(in_features=7, out_features=6)
        self.act1 = nn.ReLU()
        
    def forward(self, x):
        x = self.linear1(x)
        x = self.act1(x)
        x = self.linear2(x)
        x = self.act1(x) 
        output = self.linear3(x)
        return output
    
model = MLPClassifier(num_features=X_scl_trn.shape[1])
summary(model, input_size=(X_scl_trn.shape[1],))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Linear-1                   [-1, 18]             180
              ReLU-2                   [-1, 18]               0
            Linear-3                    [-1, 7]             133
              ReLU-4                    [-1, 7]               0
            Linear-5                    [-1, 6]              48
Total params: 361
Trainable params: 361
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.00
Forward/backward pass size (MB): 0.00
Params size (MB): 0.00
Estimated Total Size (MB): 0.00
----------------------------------------------------------------


In [11]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.AdamW(model.parameters(), lr = 0.1)
optimizer

AdamW (
Parameter Group 0
    amsgrad: False
    betas: (0.9, 0.999)
    capturable: False
    decoupled_weight_decay: True
    differentiable: False
    eps: 1e-08
    foreach: None
    fused: None
    lr: 0.1
    maximize: False
    weight_decay: 0.01
)

Generating predictions with default weights

In [12]:
y_pred = model(X_torch.float())
y_pred[:3]

tensor([[-0.1743, -0.1118, -0.0287,  0.3207, -0.2508,  0.2074],
        [-0.1457, -0.1159, -0.0247,  0.3076, -0.2559,  0.2098],
        [-0.1703, -0.1054, -0.0348,  0.3118, -0.2533,  0.2048]],
       grad_fn=<SliceBackward0>)

In [13]:
print(y_torch.shape)

torch.Size([149, 6])


In [14]:
#y_torch = y_torch.unsqueeze(1)
print(y_torch.shape)
print(y_pred.shape)

torch.Size([149, 6])
torch.Size([149, 6])


### Initial Log Loss

In [15]:
loss = criterion(y_pred, y_torch.float())
loss

tensor(1.8491, grad_fn=<DivBackward1>)

In [16]:
for epoch in np.arange(0,1000):
    for i, batch in enumerate(data_loader, 1):
      # Forward pass: Compute predicted y by passing x to the model
      y_pred_prob = model(batch[0].float())

      # Compute and print loss
      loss = criterion(y_pred_prob, batch[1].float())

      # Zero gradients, perform a backward pass, and update the weights.
      optimizer.zero_grad()

      # perform a backward pass (backpropagation)
      loss.backward()

      # Update the parameters
      optimizer.step()
    
    if epoch%10 == 0:
          print('epoch: ', epoch+1,' train loss: ', loss.item())
    #if loss.item() < 0.1:
    #    break

epoch:  1  train loss:  1.5168278217315674
epoch:  11  train loss:  0.8189548254013062
epoch:  21  train loss:  0.7209847569465637
epoch:  31  train loss:  0.8077502250671387
epoch:  41  train loss:  0.7205083966255188
epoch:  51  train loss:  0.6115272045135498
epoch:  61  train loss:  0.5379316210746765
epoch:  71  train loss:  0.7398132085800171
epoch:  81  train loss:  0.4064161777496338
epoch:  91  train loss:  0.7619203329086304
epoch:  101  train loss:  0.8782759308815002
epoch:  111  train loss:  0.6561407446861267
epoch:  121  train loss:  0.5936513543128967
epoch:  131  train loss:  0.7434694170951843
epoch:  141  train loss:  0.7992744445800781
epoch:  151  train loss:  0.7233046889305115
epoch:  161  train loss:  0.680391788482666
epoch:  171  train loss:  0.41054511070251465
epoch:  181  train loss:  0.5192486643791199
epoch:  191  train loss:  0.6631753444671631
epoch:  201  train loss:  0.5228184461593628
epoch:  211  train loss:  0.579666018486023
epoch:  221  train los

### Training Set Log Loss

In [17]:
loss

tensor(0.4403, grad_fn=<DivBackward1>)

In [18]:
X_torch_test = torch.from_numpy(X_scl_tst)
type(X_torch_test)

torch.Tensor

In [19]:
lin_output = model(X_torch_test.float()) # Equivalent predict_proba / predict
type(lin_output)

torch.Tensor

In [20]:
lin_output.size()

torch.Size([65, 6])

In [21]:
y_wt_sum = model(X_torch_test.float()) 
softmax = nn.Softmax(dim=1)
pred_proba = softmax(y_wt_sum)
pred_proba

tensor([[2.1411e-01, 2.4624e-02, 5.4067e-07, 1.5029e-23, 7.2374e-01, 3.7527e-02],
        [4.4063e-16, 3.2293e-06, 1.6834e-10, 1.0000e+00, 8.7875e-20, 2.4128e-24],
        [2.6499e-01, 7.2592e-01, 1.0128e-08, 4.2766e-22, 1.6886e-07, 9.0849e-03],
        [7.6102e-01, 5.1895e-02, 5.7551e-10, 3.2312e-28, 1.8194e-05, 1.8706e-01],
        [6.4085e-01, 2.3541e-02, 1.4440e-09, 1.0670e-27, 8.7336e-05, 3.3552e-01],
        [3.9525e-01, 5.9887e-01, 2.5424e-10, 8.0318e-27, 1.6176e-07, 5.8781e-03],
        [1.5963e-02, 4.1863e-01, 5.4470e-01, 1.6534e-03, 1.4755e-02, 4.2986e-03],
        [4.9045e-02, 9.5086e-01, 1.7901e-09, 1.8854e-23, 1.7806e-07, 9.7978e-05],
        [7.0805e-01, 7.6827e-02, 1.1063e-08, 3.2346e-25, 1.0340e-04, 2.1502e-01],
        [5.4450e-01, 4.4324e-01, 8.6028e-11, 4.3546e-28, 8.9450e-08, 1.2260e-02],
        [8.0955e-13, 2.8016e-05, 9.4125e-07, 9.9997e-01, 5.5367e-12, 3.3184e-19],
        [7.2028e-04, 9.9928e-01, 6.0537e-11, 1.2717e-19, 6.3353e-14, 1.6713e-07],
        [4.3014e

`detach().numpy()` converts `torch.Tensor` into `numpy` object

In [22]:
y_pred = np.argmax(pred_proba.detach().numpy(), axis=1 )
y_pred

array([4, 3, 1, 0, 0, 1, 2, 1, 0, 0, 3, 1, 1, 0, 1, 3, 3, 0, 2, 1, 0, 3,
       1, 1, 1, 0, 1, 0, 0, 0, 0, 3, 0, 4, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0,
       3, 3, 1, 1, 1, 1, 0, 0, 1, 0, 1, 1, 3, 0, 0, 1, 0, 0, 2, 0, 1])

In [23]:
y_test_org = np.argmax(y_test, axis=1 )

### Test Set Accuracy Score

In [24]:
print(accuracy_score(y_test_org,y_pred))

0.6923076923076923


### Test Set Log Loss

In [25]:

log_loss(y_test_org, pred_proba.detach().numpy())

1.4590490637623557